In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_news_links_with_selenium(base_url):
    options = Options()
    options.headless = False  
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(base_url)
    links = set()  # 중복을 방지하기 위해 set을 사용합니다.
    i = 0

    while True:
        time.sleep(2)  # 페이지 로딩을 기다림
        try:
            # 페이지 하단으로 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div#load-more button.button-fluid.cta-large"))
            )
            load_more_button.click()
            i += 1
            print(i)
            if i == 365:
                break
        except Exception as e:
            print(f"더 이상 '더 보기' 버튼을 찾을 수 없습니다. 에러: {e}")
            break
        
        # 매번 '더 보기' 버튼을 클릭한 후에 현재 페이지에서 링크를 수집합니다.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        news_links = soup.find_all('a', {'data-description': 'title'})
        new_links = {link['href'] for link in news_links if 'href' in link.attrs}
        links.update(new_links)
    
    driver.quit()
    return list(links)

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content_div = soup.find('div', class_='wpb_wrapper')
    sections = content_div.find_all('section')
    content = '\n'.join([section.text.strip() for section in sections])
    
    return {
        'title': title,
        'content': content,
        'url': url
    }

base_url = 'url ' #혹시 모를 저작권 문제로 url 생략하였습니다.
news_links = get_news_links_with_selenium(base_url)
print(news_links)

# data = []

# for link in news_links:
#     news_data = crawl_news_page(link)
#     if news_data:
#         data.append(news_data)

# # Create a DataFrame from the collected data
# df = pd.DataFrame(data)
# print(df.head())


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [29]:
df

,title,date,content,url
0,오늘의 5가지 이슈: 긴축베팅 과도? ECB 7월 인상 - Bloomberg Pro...,2022.04.21,(블룸버그) — 인플레이션이 피크에 가까워지고 금리 인상 베팅이 과도하다는 진단이 ...,https://www.bloomberg.co.kr/blog/five-bond-rep...
1,오늘의 5가지 이슈: 美블루스윕? 사우디감산 - Bloomberg Professio...,2021.01.06,(블룸버그) — 미국 상원의 다수당을 결정지을 조지아주 결선투표가 진행되고 있는 가...,https://www.bloomberg.co.kr/blog/five-bluesweep/
2,"오늘의 5가지 이슈: OECD도 하향, 테일리스크 - Bloomberg Profes...",2022.06.09,(블룸버그) — 뉴욕증시는 인플레이션과 그에 따른 통화정책 긴축 우려 속에 3거래일...,https://www.bloomberg.co.kr/blog/oecd-tailrisk/
3,오늘의 5가지 이슈: 인플레 피크? 되살아난 75bp - Bloomberg Prof...,2022.05.11,(블룸버그) — 뉴욕증시는 최근 며칠 사이에 1.7조 달러 가량 시가총액이 날아간 ...,https://www.bloomberg.co.kr/blog/five-inflatio...
4,"오늘의 5가지 이슈: 어닝시즌 기대, 시진핑 경고 - Bloomberg Profes...",2022.01.18,(블룸버그) — 중국을 비롯해 세계 주요 경제가 팬데믹발 침체에서 강하게 반등한 이...,https://www.bloomberg.co.kr/blog/five-earnings...
5,"오늘의 5가지 이슈: 옐런방중, 연준 의사록 - Bloomberg Professio...",2023.07.06,(블룸버그) — 미-중간 기술 전쟁이 가열되고 있는 가운데 옐런 미 재무장관이 양국...,https://www.bloomberg.co.kr/blog/five-yellen-c...
6,"오늘의 5가지 이슈: 퍼스트 리퍼블릭 불안, 연준인상 불확실 - Bloomberg ...",2023.04.26,(블룸버그) — 퍼스트 리퍼블릭 은행이 어닝쇼크로 다시 투자자 불안을 키우면서 뉴욕...,https://www.bloomberg.co.kr/blog/five-first-re...
7,"오늘 5가지 이슈: 美물가우려 일축, 日여행중단 - Bloomberg Profess...",2021.05.25,일부 연준 인사들이 미국내 경제활동이 재개되는 가운데 물가 압력의 급등세가 지속될 ...,https://www.bloomberg.co.kr/blog/five-things-t...
8,"오늘의 5가지 이슈: ‘연준은 고릴라’, 스웨덴 인하 - Bloomberg Prof...",2024.05.09,(블룸버그) — 뉴욕증시는 인텔 악재와 연준 인사의 매파적 발언 속에 5거래일만에 ...,https://www.bloomberg.co.kr/blog/fed-ecb-swede...
9,"오늘의 5가지 이슈: 침체 우려에 내년 금리인하 예상, 유가 하락 - Bloombe...",2022.07.06,,https://www.bloomberg.co.kr/blog/five-july-6/


In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

def get_news_links(base_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    i = 0
    links = []
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        news_links = soup.find_all('a', {'data-description': 'title'})
        links.extend([link['href'] for link in news_links if 'href' in link.attrs])
        print(page_number)

        load_more_div = soup.find('div', id='load-more')
        if load_more_div and '더 보기' in load_more_div.text:
            page_number += 1
            time.sleep(random.uniform(1, 1.1))  
            if page_number == 3:
                break
        else:
            break
    return links

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find('title').text.strip()

    content_div = soup.find('div', class_='wpb_wrapper')
    sections = content_div.find_all('section')
    content = '\n'.join([section.text.strip() for section in sections])
    
    return {
        'title': title,
        'content': content,
        'url': url
    }

base_url = 'url' #혹시 모를 저작권 문제로 url 생략하였습니다.
news_links = get_news_links(base_url)
print(news_links, base_url)

data = []

for link in news_links:
    news_data = crawl_news_page(link)
    if news_data:
        data.append(news_data)


df = pd.DataFrame(data)
print(df.head())


1
2
['https://www.bloomberg.co.kr/blog/five-jolts-pimco-bond/', 'https://www.bloomberg.co.kr/blog/five-ism-manufacturing-trump-bond/', 'https://www.bloomberg.co.kr/blog/five-april-pce-opec/', 'https://www.bloomberg.co.kr/blog/five-trump-guilty-cre/', 'https://www.bloomberg.co.kr/blog/five-private-credit-fed-beigebook/', 'https://www.bloomberg.co.kr/blog/five-t1-fed-hike/', 'https://www.bloomberg.co.kr/blog/five-things-to-know-155/', 'https://www.bloomberg.co.kr/blog/five-pce-korea-china-japan/', 'https://www.bloomberg.co.kr/blog/five-fed-cut-dec-cmbs/', 'https://www.bloomberg.co.kr/blog/five-fed-minutes-boe-2/', 'https://www.bloomberg.co.kr/blog/five-jolts-pimco-bond/', 'https://www.bloomberg.co.kr/blog/five-ism-manufacturing-trump-bond/', 'https://www.bloomberg.co.kr/blog/five-april-pce-opec/', 'https://www.bloomberg.co.kr/blog/five-trump-guilty-cre/', 'https://www.bloomberg.co.kr/blog/five-private-credit-fed-beigebook/', 'https://www.bloomberg.co.kr/blog/five-t1-fed-hike/', 'https://

In [4]:
import concurrent.futures
import pandas as pd
flag = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(crawl_news_page, link): link for link in final_list}
    for future in concurrent.futures.as_completed(future_to_url):
        link = future_to_url[future]
        try:
            news_data = future.result()
            if news_data:
                data.append(news_data)
        except Exception as e:
            print(f"{link}에서 데이터를 크롤링하는 중 오류 발생: {e}")

        if flag%10 == 0:
            print(flag)
        flag +=1

# 수집된 데이터로 DataFrame 생성
df = pd.DataFrame(data)
print(df.head())

0
10
https://www.mk.co.kr/news/it/7278739에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
https://www.mk.co.kr/news/economy/7279702에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
20
30
https://www.mk.co.kr/news/economy/7279763에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
40
50
60
70
80
https://www.mk.co.kr/news/business/7285926에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
https://www.mk.co.kr/news/economy/7288452에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
90
100
110
120
https://www.mk.co.kr/news/economy/7294996에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
130
https://www.mk.co.kr/news/it/7297435에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
140
150
160
170
https://www.mk.co.kr/news/economy/7304182에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
180
https://www.mk.co.kr/news/economy/7304221에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribu

In [7]:
df.to_excel('mail.xlsx')
df.to_csv('mail.csv')

In [58]:
import pandas as pd

# 데이터프레임을 엑셀 파일로 저장
df.to_excel('news_data_10_0529fdsaf1028ads.xlsx', index=False)
